In [1]:
# | default_exp commons.stats

In [43]:
from itertools import groupby

import antropy as ant
import numpy as np
import pandas as pd
import pywt
import scipy.stats as stats
import statsmodels.api as sm

# | export
import torch
from nolds import dfa, hurst_rs
from scipy.fftpack import fft
from scipy.signal import find_peaks, welch
from scipy.stats import variation
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import STL, seasonal_decompose
from statsmodels.tsa.stattools import acf, adfuller, kpss, pacf

In [44]:
# | export


# Ensure tensor input
def ensure_tensor(series):
    if isinstance(series, pd.Series):
        return torch.tensor(series.values, dtype=torch.float32)
    elif isinstance(series, np.ndarray):
        return torch.tensor(series, dtype=torch.float32)
    elif isinstance(series, torch.Tensor):
        return series.float()
    else:
        raise ValueError("Input must be a pandas Series, NumPy array, or PyTorch tensor")


# Feature extraction function


def extract_stats_features(series, max_lag=10):
    series = ensure_tensor(series)
    series_np = series.cpu().numpy()

    features = {}

    # Basic Stats
    features["mean"] = torch.mean(series).item()
    features["std"] = torch.std(series).item()
    features["var"] = torch.var(series).item()
    features["skewness"] = stats.skew(series_np)
    features["kurtosis"] = stats.kurtosis(series_np)
    features["min"] = torch.min(series).item()
    features["max"] = torch.max(series).item()
    features["range"] = features["max"] - features["min"]
    features["median"] = torch.median(series).item()
    features["iqr"] = np.percentile(series_np, 75) - np.percentile(series_np, 25)
    features["mad"] = torch.median(torch.abs(series - torch.median(series))).item()
    features["cv"] = variation(series_np)

    # Normality Tests
    _, features["shapiro_p"] = stats.shapiro(series_np)
    _, features["ks_p"] = stats.kstest(series_np, "norm")
    _, features["jarque_bera_p"] = stats.jarque_bera(series_np)

    # Stationarity Tests
    features["adf_p"] = adfuller(series_np)[1]
    features["kpss_p"] = kpss(series_np, regression="c")[1]

    # Autocorrelation
    acf_values = acf(series_np, nlags=max_lag)
    pacf_values = pacf(series_np, nlags=max_lag)
    for lag in range(1, max_lag + 1):
        features[f"acf_{lag}"] = abs(acf_values[lag])
        features[f"pacf_{lag}"] = abs(pacf_values[lag])

    # Ljung-Box Test for white noise
    features["ljungbox_p"] = abs(acorr_ljungbox(series_np, lags=[1]).iloc[0, 1])

    # Fourier Transform (Dominant Frequency)
    fft_values = np.abs(fft(series_np))
    features["fft_peak"] = np.max(fft_values)

    # Power Spectral Density (PSD) peak
    freqs, psd = welch(series_np)
    features["psd_peak"] = np.max(psd)

    # Hurst Exponent
    features["hurst_exponent"] = hurst_rs(series_np)
    features["dfa"] = dfa(series_np)

    # Sparsity Measure
    features["sparsity"] = np.sum(series_np == 0) / len(series_np)

    # Entropy Measures
    features["perm_entropy"] = ant.perm_entropy(series_np)
    features["spectral_entropy"] = ant.spectral_entropy(series_np, sf=1.0)
    features["svd_entropy"] = ant.svd_entropy(series_np)
    features["approx_entropy"] = ant.app_entropy(series_np)
    features["sample_entropy"] = ant.sample_entropy(series_np)

    # Change-point detection
    peaks, properties = find_peaks(series_np, prominence=1)
    features["num_peaks"] = len(peaks)
    features["peak_prominence_mean"] = (
        np.mean(properties["prominences"]) if len(properties["prominences"]) > 0 else 0
    )

    # Trend and Seasonality
    stl = seasonal_decompose(series_np, period=13)
    features["sd_trend_std"] = np.std(stl.trend)
    features["sd_seasonal_std"] = np.std(stl.seasonal)
    features["sd_resid_std"] = np.std(stl.resid)

    # Trend and Seasonality
    stl = STL(series_np, period=13).fit()
    features["stl_trend_std"] = np.std(stl.trend)
    features["stl_seasonal_std"] = np.std(stl.seasonal)
    features["stl_resid_std"] = np.std(stl.resid)

    # Longest flat segment
    features["longest_flat_segment"] = max(
        [len(list(g)) for k, g in groupby(series_np) if k == 0], default=0
    )

    # Wavelet Transform Features
    coeffs = pywt.wavedec(series_np, "db4", level=3)
    features["wavelet_energy"] = sum(np.sum(np.abs(c) ** 2) for c in coeffs)
    features["wavelet_entropy"] = stats.entropy(np.hstack(coeffs))

    # Higher Order Moments
    features["mean_abs_change"] = np.mean(np.abs(np.diff(series_np)))
    features["longest_positive_run"] = max(
        [sum(1 for _ in g) for k, g in groupby(series_np > 0) if k], default=0
    )

    return pd.Series(features)

In [45]:
# | hide
# Example Usage
if __name__ == "__main__":
    ts = pd.Series(np.random.randn(1000))  # Example time series
    features = extract_stats_features(ts, max_lag=10)
    print(features)

mean                     -0.006549
std                       0.955638
var                       0.913245
skewness                 -0.060189
kurtosis                 -0.035915
                           ...    
longest_flat_segment      0.000000
wavelet_energy          938.796509
wavelet_entropy               -inf
mean_abs_change           1.065341
longest_positive_run      8.000000
Length: 61, dtype: float64


/tmp/ipykernel_12920/105761769.py:42: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  features['kpss_p'] = kpss(series_np, regression='c')[1]
